**Name:**

**Student ID:**

In this practical assignment, we will develope a U-Net for semantis segmentation of lung CT-scans.

For this task, you will use a dataset consisting of CT-scans from lungs of COVID-19 patients. The regions affected by COVID-19 are marked with labels of ground glass opacity. You will be training and testing the U-Net model on this dataset.

## Imports

In [ ]:
# Import any needed packages here
import torch
import numpy as np
from matplotlib import pyplot as plt

## Prepare Data
Now download and unzip the data. (Note: If you face any problems for downloading uzing gdown and unzipping, you may also download the data manually using the Goggle Drive links in the code)

In [ ]:
from zipfile import ZipFile
import os
import numpy as np
import gdown

output = 'images.zip'
url= "https://drive.google.com/file/d/1UYI8pNkuFqcOyQtnsMMXwbh92GuiMIfo/view?usp=sharing"
gdown.download(url, output, quiet=True)
output = 'masks.zip'
url= "https://drive.google.com/file/d/1mYB32-cZwZEUyHogNZv-Y4eaPbRxXaHt/view?usp=sharing"
gdown.download(url, output, quiet=True)

In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("images.zip", "r")
zip_ref.extractall()
zip_ref.close()

zip_ref = zipfile.ZipFile("masks.zip", "r")
zip_ref.extractall()
zip_ref.close()
images = np.load('images_medseg.npy').astype(np.float32)
masks = np.load('masks_medseg.npy').astype(np.float32)

In [ ]:
print("Images Shape:", images.shape)
print("Masks Shape:", masks.shape)

The dataset consists of 100 CT images with size 512*512. In the Masks data,for each image 4 masks are provided with different meanings. The first mask (index 0) is the ground glass opacity class, indicating the regions affected by COVID-19. Also, the last mask (index 4) is the background class.

You can see the visulaiztion of a few of the CT images, and their given masks using the code below. In this assignment, use the first mask (ground glass opacity) for the segmentation task (You may also test it on other classes if you like too!)

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("dark_background")

def visualize(image_batch, mask_batch=None, pred_batch=None, num_samples=8):
    num_classes = mask_batch.shape[-1] if mask_batch is not None else 0
    fix, ax = plt.subplots(num_classes + 1, num_samples, figsize=(num_samples * 2, (num_classes + 1) * 2))
    for i in range(num_samples):
        ax_image = ax[0, i] if num_classes > 0 else ax[i]
        ax_image.imshow(image_batch[i,:,:,0], cmap="Greys")
        ax_image.set_xticks([]) 
        ax_image.set_yticks([])
        
        if mask_batch is not None:
            for j in range(num_classes):
                if pred_batch is None:
                    #print(j)
                    mask_to_show = mask_batch[i,:,:,j]
                else:
                    mask_to_show = np.zeros(shape=(*mask_batch.shape[1:-1], 3)) 
                    mask_to_show[..., 0] = pred_batch[i,:,:,j] > 0.5
                    mask_to_show[..., 1] = mask_batch[i,:,:,j]
                ax[j + 1, i].imshow(mask_to_show, vmin=0, vmax=1)
                ax[j + 1, i].set_xticks([]) 
                ax[j + 1, i].set_yticks([]) 

    plt.tight_layout()
    plt.show()


visualize(images, masks)

## Create Your Dataset Class (10%)

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):

  ## TODO: Implement your dataset here (you may implement it based on torch.utils.data.Dataset).
  return None

# TODO: Instatntiate your dataset here
dataset = Dataset(None)

## Split Dataset and Dataloader (5%)

In this part, split the dataset (80% train, 10% validation, 10% test). Then, create your train and validation DataLoader, using random shuffle and batch size of 4.

In [ ]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader

# TODO: Split the train and validation data and create dataloaders

## Implement your U-Net (45%)

In this section implement your U-Net model, inherited from torch.nn.Module.

In the next text cell, explain the architecture of the implemented U-Net. Then, complete the code cell to creat your class.

Explain Here:

In [ ]:
class UNet(nn.Module):
    def __init__(self, channels_in, num_classes):
        super(UNet, self).__init__()
        # TODO: Initialize the model
        

    def forward(self, x):
      # TODO: Implement the forward function  
      return None

## IOU (5%)
In this section, implement the function calculating IOU (Intersection Over Union) for a given prediction (your model's output) and target (ground-truth labels).

In [ ]:
def IoU(output, target):
  # TODO: Implement IOU
  Return None

## Define your model, optimizer, and loss function (5%)

Instantiate your model. Also, set your optimizer to Adam optimizer, and your criterion to Bincry Cross-Entropy Loss.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = None
model.to(device)
optimizer = None
criterion = None

## Train and test function (25%)

Train your model for a desired number of epochs (at least 15). At each epoch, run your model on the validation data (reporting the loss and IOU value), and save the model which has the best average IOU on the validation set. Finally, report the average IOU on the target data.

During training, also plot the train loss values, average IOU on the train data, and avergae IOU on the validation data per epochs.

In [ ]:
def train(model, train_dataloader, validation_dataloader, num_epochs):
    # TODO: Implement your train function here.


def test(model, test-dataloader):
    # TODO: Implement your test function here.  
            

In [ ]:
# TODO: run the train and test functions and draw the required outputs.
# To get the full point from this part, you should get IOU above 0.9

## Visualize the models output on 4 of target images (5%)

Run the model on 4 samples from the target set. Visualize the outcome by plotting the groud-truth labels, model's predicition, and the difference between them.

In [ ]:
# TODO: Run the model on 4 samples from target set, Visualize the labels and predictions, and compare them

## Improve U-Net (Bonus) (10%)

In this bnous part, you can add your ideas for improving your model's performance. As an idea, you can search about exploiting IOU metric in the loss function. (you may use other ideas as well!) 

In [ ]:
# TODO: Improve U-NET